In [30]:
%pylab
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import scipy
import pandas as pd
import seaborn as sns
path =  (os.path.abspath(os.path.join(r'.', os.pardir)))
import sys
sys.path.append(path)
from nuclei_segmentation import otsu as ot
from nuclei_segmentation import dicescore as dsc
from PIL import Image
import os
import os.path

Using matplotlib backend: QtAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\ievas\anaconda3\envs\dataanalysis0104_env\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [31]:
# Set ground truth images from N2DH-GOWT1 dataset and N2DL-HeLa dataset to 0 and 1

def binarize(x):
    img = x.copy()

    for o in np.ndindex(img.shape):
        if img[o] > 0: 
           img[o] = 1
        else:
            img[o] = 0
    
    return img

## NIH3T3 Dataset

In [32]:
# Load raw images of NIH3T3-dataset

img_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/img")
for f in os.listdir(path):
    img_NIH3T3.append(imread(os.path.join(path , f)))

# Load ground truth images of NIH3T3-dataset
 
gt_NIH3T3 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
for f in os.listdir(path):
    gt_NIH3T3.append(imread(os.path.join(path , f)))

In [33]:
# Make a copy of all images of each dataset / use as "no preprocessed images"
copy_img_NIH3T3 = img_NIH3T3.copy()

In [34]:
print(gt_NIH3T3)
shape(gt_NIH3T3)

[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0

(18, 1024, 1344)

In [35]:
print(img_NIH3T3)

[array([[0.14509805, 0.15294118, 0.14901961, ..., 0.07843138, 0.07843138,
        0.07450981],
       [0.16078432, 0.14901961, 0.14901961, ..., 0.07843138, 0.08235294,
        0.08235294],
       [0.14901961, 0.14901961, 0.15686275, ..., 0.08235294, 0.08235294,
        0.07843138],
       ...,
       [0.08627451, 0.08235294, 0.08627451, ..., 0.09411765, 0.09019608,
        0.09019608],
       [0.08627451, 0.09019608, 0.08627451, ..., 0.09411765, 0.09019608,
        0.09411765],
       [0.08627451, 0.08235294, 0.08627451, ..., 0.09411765, 0.09411765,
        0.09019608]], dtype=float32), array([[0.08235294, 0.08235294, 0.08235294, ..., 0.12941177, 0.1254902 ,
        0.13333334],
       [0.07843138, 0.08235294, 0.08627451, ..., 0.12941177, 0.13333334,
        0.1254902 ],
       [0.08235294, 0.08627451, 0.08235294, ..., 0.12941177, 0.12941177,
        0.13333334],
       ...,
       [0.13333334, 0.13333334, 0.12156863, ..., 0.08627451, 0.08235294,
        0.08235294],
       [0.14901961

#### Median filter

In [36]:
# Apply median filter to all images of each dataset
# mit kleinerern Filtern läuft es schneller durch
# filter größen die man möchte
filter = [3, 4, 5, 10, 15, 20]
median_img_NIH3T3 = []
for j in range (0, len(filter)):
    
    for i in range(0, len(copy_img_NIH3T3)):
        median_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[j]))
    print("filtersize done")    

filtersize done
filtersize done
filtersize done
filtersize done
filtersize done
filtersize done


In [37]:
#mit 2 D array geht so aber-> otsu nicht mehr möglich


# Apply median filter to all images of each dataset
# mit kleinerern Filtern läuft es schneller durch
# filter größen die man möchte
#filter = [3, 4, 5]
#median3_img_NIH3T3 = []
#median4_img_NIH3T3 = []
#median5_img_NIH3T3 = []

#for j in range (0, len(filter)):
   # median_img_NIH3T3 = []
    #for i in range(0, len(copy_img_NIH3T3)):
    #    median3_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[0]))
    #    median4_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[1]))
    #    median5_img_NIH3T3.append(scipy.ndimage.median_filter(copy_img_NIH3T3[i] , size= filter[2]))
    #median_img_NIH3T3.append(median3_img_NIH3T3) 
    #median_img_NIH3T3.append(median4_img_NIH3T3)
    #median_img_NIH3T3.append(median5_img_NIH3T3)

    #print("filtersize done")  

#### gaussian filter

In [38]:
#shape(copy_img_NIH3T3)
shape(median_img_NIH3T3)

(108, 1024, 1344)

In [39]:
# Apply gaussian filter to all images of each dataset
#filter größen die mach möchte 
filter_g = [3, 4, 5, 10, 15, 20]
gauss_img_NIH3T3 = []

for j in range (0, len(filter_g)):
    for i in range(0,len(copy_img_NIH3T3)):
        gauss_img_NIH3T3.append(scipy.ndimage.gaussian_filter(copy_img_NIH3T3[i] , filter_g[j]))
    print("filtersize done")
        


filtersize done
filtersize done
filtersize done
filtersize done
filtersize done
filtersize done


In [40]:
shape(gauss_img_NIH3T3)
shape(median_img_NIH3T3)

(108, 1024, 1344)

## Otsu thresholding

In [41]:
# Apply otsu thresholding to all images of each dataset - median filter
otsu_median_img_NIH3T3 = []
for i in range(0,len(median_img_NIH3T3)):
    otsu_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [i] , 256))
print(shape(otsu_median_img_NIH3T3))
print(otsu_median_img_NIH3T3)




(108, 1024, 1344)
[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [

In [42]:
#otsu median mit 2D array-> nicht möglich

#otsu_median_img_NIH3T3 = []
#otsu3_median_img_NIH3T3 = []
#otsu4_median_img_NIH3T3 = []
#otsu5_median_img_NIH3T3 = []

#for i in range(0,len(median_img_NIH3T3)):
    #otsu3_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [0] , 256))
    #otsu4_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [1] , 256))
    #otsu5_median_img_NIH3T3.append(ot.otsu_thresholding(median_img_NIH3T3 [2] , 256))
#otsu_median_img_NIH3T3.append(otsu3_median_img_NIH3T3)
#otsu_median_img_NIH3T3.append(otsu4_median_img_NIH3T3)
#otsu_median_img_NIH3T3.append(otsu5_median_img_NIH3T3)



In [43]:
# Apply otsu thresholding to all images of each dataset - gaussian filter
otsu_gauss_img_NIH3T3 = []
for i in range(0,len(gauss_img_NIH3T3)):
    otsu_gauss_img_NIH3T3.append(ot.otsu_thresholding(gauss_img_NIH3T3[i] , 256))


In [48]:
gt6_NIH3T3 = []

path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/NIH3T3/gt")
y=0
while y<6:
    for f in os.listdir(path):
        gt6_NIH3T3.append(imread(os.path.join(path , f)))
    y+=1

print(shape(gt6_NIH3T3))

(108, 1024, 1344)


## Dice score

In [49]:
# Dice score - NIH3T3-dataset
## median filter
dice_otsu_median_img_NIH3T3 = []

for i in range(0, len(otsu_median_img_NIH3T3)):
  dice_otsu_median_img_NIH3T3.append(dsc.dice(otsu_median_img_NIH3T3[i] , gt6_NIH3T3[i]))

print(dice_otsu_median_img_NIH3T3)

### Es gibt mir zwar 18 mal 6 werte aus (also für die 6 filtersizes die ich gewählt habe)
# aber die werte stimmen nicht... wenn man sie mit "window_preprocessing_short.ipynb" vergleicht wo man die richtige
# dice score werte sieht für NIH3T3 für eine filtersize von 5 was hier also bei reihe  7 anfangen würde...
# sieht so aus als würde er nur das erste bild korrekt dice scoren (0.913...) und danach kommt nur noch doofes zeug raus...

##dice_otsu_median_img_NIH3T3 = []
#for i in range (0, len (gt6_NIH3T3)):
   #for j in range( 0, len(otsu_median_img_NIH3T3)):
     #dice_otsu_median_img_NIH3T3.append(dsc.dice(otsu_median_img_NIH3T3[j] , gt6_NIH3T3[i]))
#print(dice_otsu_median_img_NIH3T3)



[0.9131440698604878, 0.8853711103603341, 0.8298069079796091, 0.7362834954173372, 0.7534219447587042, 0.6733729284060831, 0.5122585466957399, 0.7259260821045561, 0.034365388558540086, 0.46442334002188984, 0.0, 0.6826525572468858, 0.00021072595090085343, 0.575303696970605, 0.6287369454420759, 0.06277345147354228, 0.06692755965800332, 0.7980940984936302, 0.8977948217808237, 0.8880969086439946, 0.8185685735865199, 0.739506084579049, 0.7564891645225117, 0.6760668227487759, 0.5681786876807864, 0.7012981734910003, 0.03489157810676331, 0.4664764662545974, 0.0, 0.6836885181690923, 0.00014924521427662162, 0.5767613169748401, 0.6162828660365994, 0.06835391087246485, 0.06880794224744917, 0.7981533689836506, 0.8946808298799254, 0.8849869001333296, 0.8300675280378401, 0.7357748411841214, 0.7529539648467392, 0.6170388301551231, 0.6099779516517357, 0.6973465117594204, 0.03084486966478485, 0.4631152690677724, 0.0, 0.6824470203713232, 0.00014050370578524009, 0.5739847098266359, 0.6282657339684911, 0.065

In [ ]:
print(len(gt_NIH3T3)) #<- muss irgendwie auch 108 lang, bild muss sich quasi immer wiederhoen
print(len(gt6_NIH3T3))
print(len(median_img_NIH3T3))

18
6
108


In [ ]:
print(otsu_median_img_NIH3T3)

[array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0

In [ ]:
# Dice score - N2DH-GOWT1-dataset
## gaussian filter

dice_otsu_gauss_img_N2DH_GOWT1 = []
for j in range(0,len(otsu_gauss_img_N2DH_GOWT1)):
    dice_otsu_gauss_img_N2DH_GOWT1.append(dsc.dice(otsu_gauss_img_N2DH_GOWT1[j] , binary_gt_N2DH_GOWT1[j]))

print(dice_otsu_gauss_img_N2DH_GOWT1)

NameError: name 'otsu_gauss_img_N2DH_GOWT1' is not defined

## N2DH-GOWT1

In [ ]:
# Load raw images from N2DH-GOWT1-dataset

img_N2DH_GOWT1 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)), r"data/Otsu_data/N2DH-GOWT1/img")
for f in os.listdir(path):
    img_N2DH_GOWT1.append(imread(os.path.join(path , f)))


# Load ground truth images from N2DH-GOWT1-dataset

gt_N2DH_GOWT1 = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)), r"data/Otsu_data/N2DH-GOWT1/gt")
for f in os.listdir(path):
    gt_N2DH_GOWT1.append(imread(os.path.join(path , f)))

In [ ]:
# binarize

binary_gt_N2DH_GOWT1 = []
for i in range(0,len(gt_N2DH_GOWT1)):
    binary_gt_N2DH_GOWT1.append(binarize(gt_N2DH_GOWT1[i])) 

In [ ]:
#copy
copy_img_N2DH_GOWT1 = img_N2DH_GOWT1.copy()

In [ ]:
# gaussian filter 

filter_g = [3, 4, 5, 10, 15, 20]
gauss_img_N2DH_GOWT1 = []

for j in range (0, len(filter_g)):

    for i in range(0,len(copy_img_N2DH_GOWT1)):
        gauss_img_N2DH_GOWT1.append(scipy.ndimage.gaussian_filter(copy_img_N2DH_GOWT1[i] , filter_g[j] ))


In [ ]:
# otsu
otsu_gauss_img_N2DH_GOWT1 = []
for i in range(0,len(gauss_img_N2DH_GOWT1)):
    otsu_gauss_img_N2DH_GOWT1.append(ot.otsu_thresholding(gauss_img_N2DH_GOWT1 [i] , 256))


In [ ]:
#dice





## N2DL-HeLa

In [ ]:
# Load raw images from N2DL-HeLa-dataset

img_N2DL_HeLa = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)), r"data/Otsu_data/N2DL-HeLa/img")
for f in os.listdir(path):
    img_N2DL_HeLa.append(imread(os.path.join(path , f)))


# Load ground truth images from N2Dl-HeLa-dataset

gt_N2DL_HeLa = []
path = os.path.join(os.path.abspath(os.path.join(r'.', os.pardir)),r"data/Otsu_data/N2DL-HeLa/gt")
for f in os.listdir(path):
    gt_N2DL_HeLa.append(imread(os.path.join(path , f)))

In [ ]:
binary_gt_N2DL_HeLa = []
for i in range(0,len(gt_N2DL_HeLa)):
    binary_gt_N2DL_HeLa.append(binarize(gt_N2DL_HeLa[i])) 

In [ ]:
copy_img_N2DL_HeLa = img_N2DL_HeLa.copy()

### median

In [ ]:
filter_g = [3, 4, 5, 10, 15, 20]
median_img_N2DL_HeLa = []

for j in range (0, len(filter_g[j])):

    for i in range(0,len(copy_img_N2DL_HeLa)):
        median_img_N2DL_HeLa.append(scipy.ndimage.median_filter(copy_img_N2DL_HeLa[i] , filter_g[j]))




## otsu

In [ ]:
otsu_median_img_N2DL_HeLa = []
for i in range(0,len(median_img_N2DL_HeLa)):
    otsu_median_img_N2DL_HeLa.append(ot.otsu_thresholding(median_img_N2DL_HeLa [i] , 256))